In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "",
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/<>/oauth2/token"}


dbutils.fs.mount(
source = "abfss://hcm-traffic-data@realtrafficanalysisdata.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/hcm-traffic",
extra_configs = configs)
  

True

In [0]:
%fs
ls "/mnt/hcm-traffic/raw"

path,name,size,modificationTime
dbfs:/mnt/hcm-traffic/raw/nodes.csv,nodes.csv,19444470,1699606587000
dbfs:/mnt/hcm-traffic/raw/segment_status.csv,segment_status.csv,3579383,1699606615000
dbfs:/mnt/hcm-traffic/raw/segments.csv,segments.csv,10287487,1699606600000
dbfs:/mnt/hcm-traffic/raw/streets.csv,streets.csv,201060,1699606642000


In [0]:
# 

In [0]:
nodes = spark.read.format("csv").option("inferSchema", "true").option("header", "true").load("/mnt/hcm-traffic/raw/nodes.csv")
segment_status = spark.read.format("csv").option("inferSchema", "true").option("header", "true").load("/mnt/hcm-traffic/raw/segment_status.csv")
segments = spark.read.format("csv").option("inferSchema", "true").option("header", "true").load("/mnt/hcm-traffic/raw/segments.csv")
streets = spark.read.format("csv").option("inferSchema", "true").option("header", "true").load("/mnt/hcm-traffic/raw/streets.csv")

In [0]:
print(nodes.show(2))
print(nodes.describe().show())

+---------+-----------+----------+
|      _id|       long|       lat|
+---------+-----------+----------+
|366367223|106.6290559|10.8042433|
|366367233|106.7097014|10.7711099|
+---------+-----------+----------+
only showing top 2 rows

None
+-------+--------------------+-------------------+--------------------+
|summary|                 _id|               long|                 lat|
+-------+--------------------+-------------------+--------------------+
|  count|              577967|             577967|              577967|
|   mean| 4.315268887282453E9|   106.703699854799|   10.80833211476907|
| stddev|1.8879835245470002E9|0.06966474231964309|0.049735991885667664|
|    min|           366367223|        106.5360046|          10.6760012|
|    max|          6205235785|        106.8679941|          10.8889982|
+-------+--------------------+-------------------+--------------------+

None


In [0]:
segment_status.show(2)
segment_status.describe().show()

+---+--------------------+----------+--------+
|_id|          updated_at|segment_id|velocity|
+---+--------------------+----------+--------+
|  0|2020-07-03 14:55:...|     24845|      20|
|  1|2020-07-03 15:02:...|     33923|      10|
+---+--------------------+----------+--------+
only showing top 2 rows

+-------+-----------------+------------------+-----------------+
|summary|              _id|        segment_id|         velocity|
+-------+-----------------+------------------+-----------------+
|  count|            90938|             90938|            90938|
|   mean|          45468.5| 40376.03000945699|20.75484395962084|
| stddev|26251.68372695359|22415.108133187623|19.56711229934773|
|    min|                0|                26|                0|
|    max|            90937|             84535|              200|
+-------+-----------------+------------------+-----------------+



In [0]:
segments.show(2)
segments.describe().show()

+---+--------------------+--------------------+----------+----------+------+---------+------------+------------+-----------+-----------+
|_id|          created_at|          updated_at| s_node_id| e_node_id|length|street_id|max_velocity|street_level|street_name|street_type|
+---+--------------------+--------------------+----------+----------+------+---------+------------+------------+-----------+-----------+
|  0|2020-10-18 13:26:...|2020-10-18 13:26:...| 373543511|5468660805|   114| 31096786|        80.0|           1|  Quốc Lộ 1|      trunk|
|  1|2020-10-18 13:26:...|2020-10-18 13:26:...|5468660805|5738158916|     9| 31096786|        80.0|           1|  Quốc Lộ 1|      trunk|
+---+--------------------+--------------------+----------+----------+------+---------+------------+------------+-----------+-----------+
only showing top 2 rows

+-------+------------------+--------------------+--------------------+-----------------+--------------------+------------------+------------------+------

In [0]:
streets.show(5)
streets.describe().show()

+--------+-----+------------+--------------+------------+
|     _id|level|max_velocity|          name|        type|
+--------+-----+------------+--------------+------------+
|31096786|    1|          80|     Quốc Lộ 1|       trunk|
|32575737|    4|        NULL|          NULL|unclassified|
|32575794|    4|        NULL|    Chu Văn An|unclassified|
|32575820|    4|        NULL| Nguyễn Văn Bá|    tertiary|
|32575823|    4|        NULL|Nguyễn Thị Nhỏ|    tertiary|
+--------+-----+------------+--------------+------------+
only showing top 5 rows

+-------+-------------------+------------------+------------------+---------+------------+
|summary|                _id|             level|      max_velocity|     name|        type|
+-------+-------------------+------------------+------------------+---------+------------+
|  count|               5553|              5553|               987|     3948|        5553|
|   mean|3.457454960606879E8| 3.199711867459031|58.232016210739616|     NULL|        NULL

In [0]:
from pyspark.sql.functions import col, monotonically_increasing_id, date_format, avg, sum
import pyspark.sql.functions as F

import pandas as pd

In [0]:
merged_df = segment_status.join(segments, segment_status['segment_id'] == segments['_id'], 'inner')\
    .select(segment_status['segment_id'], segments['created_at'], segment_status['updated_at'], segment_status['velocity'], segments['max_velocity'], \
        segments['s_node_id'], segments['e_node_id'], segments['length'], segments['street_id'], segments['street_level'], segments['street_name'], segments['street_type'])
# merged_df = merged_df.dropDuplicates(['street_id'])
merged_df = merged_df.join(nodes, merged_df['s_node_id'] == nodes['_id'].alias('node_id'), 'left')
merged_df = merged_df.drop(nodes['_id'])

merged_df = merged_df.join(streets, merged_df['street_id'] == streets['_id'], 'left')
merged_df = merged_df.drop(streets['_id'], streets['max_velocity'], streets['name'])

# add index column
merged_df = merged_df.withColumn("_id", monotonically_increasing_id())
columns = merged_df.columns
columns.insert(0, columns.pop(columns.index('_id')))
merged_df = merged_df.select(columns)


merged_df.show(10)

+---+----------+--------------------+--------------------+--------+------------+---------+----------+------+---------+------------+-------------+-----------+-----------+----------+-----+---------+
|_id|segment_id|          created_at|          updated_at|velocity|max_velocity|s_node_id| e_node_id|length|street_id|street_level|  street_name|street_type|       long|       lat|level|     type|
+---+----------+--------------------+--------------------+--------+------------+---------+----------+------+---------+------------+-------------+-----------+-----------+----------+-----+---------+
|  0|        26|2020-10-18 13:26:...|2021-04-16 00:55:...|     117|        NULL|366428456| 366416066|   116| 32575820|           4|Nguyễn Văn Bá|   tertiary|106.7687324|10.8415062|    4| tertiary|
|  1|        33|2020-10-18 13:26:...|2020-08-03 00:23:...|      27|        NULL|366469460|3792257828|    26| 32575862|           3|   Đường số 5|  secondary|106.7619572|  10.87865|    3|secondary|
|  2|        33

In [0]:
merged_df.describe().show()

+-------+-------------------+------------------+-----------------+------------------+--------------------+--------------------+------------------+--------------------+------------------+-----------+-----------+-------------------+-------------------+------------------+------------+
|summary|                _id|        segment_id|         velocity|      max_velocity|           s_node_id|           e_node_id|            length|           street_id|      street_level|street_name|street_type|               long|                lat|             level|        type|
+-------+-------------------+------------------+-----------------+------------------+--------------------+--------------------+------------------+--------------------+------------------+-----------+-----------+-------------------+-------------------+------------------+------------+
|  count|              90938|             90938|            90938|              8618|               90938|               90938|             90938|     

In [0]:
avg_velocity_per_street = merged_df.groupBy('street_name').agg(avg('velocity').alias('avg_velocity'))
avg_velocity_per_street.show()

+--------------------+------------------+
|         street_name|      avg_velocity|
+--------------------+------------------+
|          Đường số 9| 19.88888888888889|
|            Đồng Đen|             15.75|
|      Trần Đại Nghĩa| 34.07142857142857|
|      Nguyễn Văn Lạc|              13.7|
|          Lê Văn Chí|              22.9|
|           Linh Đông|16.692307692307693|
|       Hoàng Văn Thụ| 7.369555249885374|
|          Thành Thái|20.536800785083415|
|      Trần Văn Quang| 19.98840579710145|
|            Trần Quý|19.648979591836735|
|         Cầu Băng Ky|              39.0|
|      Đường tỉnh 743|              36.5|
|      Dương Thị Mười|46.666666666666664|
|     Nguyễn Thị Đặng|49.285714285714285|
|        Nguyễn Tuyển|              19.4|
|Cầu vượt Ngã tư T...|              17.0|
|         Võ Văn Kiệt|12.254259501965924|
|      Đào Trinh Nhất|21.333333333333332|
|          Minh Phụng|19.313953488372093|
|       Trần Văn Giàu|             23.08|
+--------------------+------------

In [0]:
total_length_per_street = merged_df.groupBy('street_name').agg(sum('length').alias('total_length'))
total_length_per_street.show()

+--------------------+------------+
|         street_name|total_length|
+--------------------+------------+
|          Đường số 9|         228|
|            Đồng Đen|        1254|
|      Trần Đại Nghĩa|        2496|
|      Nguyễn Văn Lạc|         180|
|          Lê Văn Chí|        2018|
|           Linh Đông|         382|
|       Hoàng Văn Thụ|       95025|
|          Thành Thái|       42655|
|      Trần Văn Quang|       12199|
|            Trần Quý|       15940|
|         Cầu Băng Ky|          89|
|      Đường tỉnh 743|          26|
|      Dương Thị Mười|          47|
|     Nguyễn Thị Đặng|         880|
|        Nguyễn Tuyển|         350|
|Cầu vượt Ngã tư T...|         393|
|         Võ Văn Kiệt|       40514|
|      Đào Trinh Nhất|        2053|
|          Minh Phụng|       16090|
|       Trần Văn Giàu|        1169|
+--------------------+------------+
only showing top 20 rows



In [0]:
street_info_df = avg_velocity_per_street.join(total_length_per_street, avg_velocity_per_street['street_name'] == total_length_per_street['street_name'])

street_info_df = street_info_df.drop(total_length_per_street['street_name'])

street_info_df = street_info_df.join(streets, street_info_df['street_name'] == streets['name'], 'left')
street_info_df = street_info_df.select('_id', 'street_name', 'total_length', 'avg_velocity', 'max_velocity', 'type')
street_info_df.show(5)

+---------+-----------+------------+-----------------+------------+------------+
|      _id|street_name|total_length|     avg_velocity|max_velocity|        type|
+---------+-----------+------------+-----------------+------------+------------+
|462968808| Đường số 9|         228|19.88888888888889|          40|    tertiary|
| 32578171| Đường số 9|         228|19.88888888888889|        NULL|unclassified|
|377308634|   Đồng Đen|        1254|            15.75|        NULL|    tertiary|
|377308631|   Đồng Đen|        1254|            15.75|        NULL|    tertiary|
|223642675|   Đồng Đen|        1254|            15.75|        NULL|    tertiary|
+---------+-----------+------------+-----------------+------------+------------+
only showing top 5 rows



In [0]:
street_info_df.describe().show()

+-------+--------------------+-----------+------------------+------------------+------------------+------------+
|summary|                 _id|street_name|      total_length|      avg_velocity|      max_velocity|        type|
+-------+--------------------+-----------+------------------+------------------+------------------+------------+
|  count|                2954|       3086|              3086|              3086|               660|        2954|
|   mean| 3.042786202728504E8|       18.0|15581.941672067402|26.733217242600144| 58.99242424242424|        NULL|
| stddev|1.9512756161963928E8|       NULL|31440.292451025296|12.720853874038998|15.904922464626624|        NULL|
|    min|            31096786|         18|                 2|               1.0|                10|    motorway|
|    max|           658328101|     Ấp Bắc|            209379|              89.5|               120|unclassified|
+-------+--------------------+-----------+------------------+------------------+----------------

In [0]:
merged_df.repartition(10).write.mode("overwrite").option("header","true").csv("/mnt/hcm-traffic/transformed/traffics")
street_info_df.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/hcm-traffic/transformed/streets")